# Instructions

1. Create a .zip file containing the **table of questions** and **all referenced images.**
  - Easiest method is to select your Google sheet & images, then right-click and download
  - Table file can be delimiter-separated values (.csv, .tsv) or excel format (.xlsx, first sheet will be used)
  - See [UQF definition](https://docs.google.com/spreadsheets/d/1-23EUnUdjSUW0MOKgnE1u-E1a4_xPn3wsTlRCSUagQ8/edit?usp=sharing) for details & [Example UQF](https://drive.google.com/file/d/1ms0c1LQf1zG7-pYDGUIZ8WJ2FXeL5hsE/view?usp=share_link) for a sample file for testing
2. Upload the .zip file to your **OWN** Google drive.
  - Ensure you are **NOT** on your batch drive account
3. Change options below to match .zip file & output requirements
4. On the menu bar (top of the page) click "Runtime" and "Run all"
5. When the dialog box appears, allow the script to access your google drive.
6. Wait for UQF to be processed (should take about 2 min)
7. Once complete, go back to your drive and download the Q & QnA PDFs created next to the uploaded UQF file.
8. To export another UQF file, start again from step 1.

In [ ]:
#@title # Options
#@markdown ### Basic options
folder_name = "uqf" #@param {type:"string"}
#@markdown - Drive folder where your .zip is located; leave blank if file is in root of your drive
uqf_zip_file_name = "uqf_example.zip" #@param {type:"string"}
#@markdown - Name of your .zip file
title = "Example Questions" #@param {type:"string"}
#@markdown - Title you want to be displayed in PDF; leave blank to use filename as title
#@markdown ---
#@markdown ### PDF encryption
#@markdown Provide passwords for viewing & editing, or allow script to generate them automatically.
password_protect = True #@param {type:"boolean"}
view_password = "" #@param {type:"string"}
#@markdown - Password to view PDF; leave blank to generate 8 character password (will be output to .txt file)
edit_password = "" #@param {type:"string"}
#@markdown - Password to edit PDF; leave blank to generate 50 character password (will be discarded for security)
#@markdown ---
#@markdown ### Additional options
verify_zip = True #@param {type:"boolean"}
#@markdown - Runs checks on files inside .zip; may be helpful to turn off if critical errors appear
format_tolerance = "strict" #@param ["strict", "lenient"]
#@markdown - Sets tolerance for option detection; may be helpful to set to 'lenient' if there are human errors in formatting
randomise_options = False #@param {type:"boolean"}
#@markdown - Randomises option order; helpful if you are generating a custom paper for your own practice

In [ ]:
#@title Connect to GDrive
from google.colab import drive

drive_path = '/content/drive'
drive.mount(drive_path)

In [ ]:
#@title Install dependencies
!pip install weasyprint==52.5
!pip install rpy2==3.5.1
!pip install pikepdf==2.11.1
!wget https://raw.githubusercontent.com/thammatthew/UQF/main/UQF.R -O "UQF.R"
!wget https://raw.githubusercontent.com/thammatthew/UQF/main/styles.css -O "styles.css"

In [ ]:
#@title Import libraries and set variables
from weasyprint import HTML, CSS
import os

local_path = os.path.join(drive_path, 'MyDrive', folder_name)
uqf_zip_file_path = os.path.join(local_path, uqf_zip_file_name)
!cp "$uqf_zip_file_path" "$uqf_zip_file_name"

%load_ext rpy2.ipython

In [ ]:
#@title Convert UQF to HTML
%%R -i uqf_zip_file_name,verify_zip,format_tolerance,randomise_options,title
install.packages("commonmark")
source("UQF.R")
uqf <- uqf_to_html(uqf_zip_file_name, verify = verify_zip, mode = format_tolerance, randomise_op = randomise_options, title = title)

In [ ]:
#@title Convert HTML to PDF
from weasyprint import HTML, CSS
import os
css = CSS("styles.css")

q_html_filename = os.path.splitext(uqf_zip_file_name)[0]+" - Questions.html"
q_pdf_filename = os.path.splitext(uqf_zip_file_name)[0]+" - Questions.pdf"
q_pdf_path = os.path.join(local_path, q_pdf_filename)
html = HTML(q_html_filename)
html.write_pdf(q_pdf_path, stylesheets = [css])

qna_html_filename = os.path.splitext(uqf_zip_file_name)[0]+" - Answers.html"
qna_pdf_filename = os.path.splitext(uqf_zip_file_name)[0]+" - Answers.pdf"
qna_pdf_path = os.path.join(local_path, qna_pdf_filename)
html = HTML(qna_html_filename)
html.write_pdf(qna_pdf_path, stylesheets = [css])

In [ ]:
#@title Password protect PDF
if password_protect:
  import secrets
  import string

  charset = string.ascii_letters + string.digits + string.punctuation
  if not view_password:
    view_password = ''.join(secrets.choice(charset) for i in range(8))
  if not edit_password:
    edit_password = ''.join(secrets.choice(charset) for i in range(50))

  import pikepdf
  from pikepdf import Pdf

  permissions = pikepdf.Permissions(
      accessibility = False,
      extract = False,
      modify_annotation = False,
      modify_assembly = False,
      modify_form = False,
      modify_other = False,
      print_highres = False,
      print_lowres = False
  )
  encryption = pikepdf.Encryption(
      user = view_password,
      owner = edit_password,
      allow = permissions,
      metadata = True
  )
      
  q_pdf = Pdf.open(q_pdf_path, allow_overwriting_input = True)
  q_pdf.save(q_pdf_path, encryption = encryption)

  qna_pdf = Pdf.open(qna_pdf_path, allow_overwriting_input = True)
  qna_pdf.save(qna_pdf_path, encryption = encryption)

  pwd_filename = os.path.splitext(uqf_zip_file_name)[0]+" - Password.txt"
  pwd_path = os.path.join(local_path, pwd_filename)

  print(view_password)

  with open(pwd_path, 'w') as f:
      f.write(view_password)

  view_password = ""
  edit_password = ""

# Help & credits

- Direct any queries/bug reports to Matthew Tham (@thammatthew on Tele)
- All code (this notebook, R script & CSS stylesheet) available at https://github.com/thammatthew/UQF